# Checkers

## Rules
<ul>
<li>Two players</li>
<li>8 by 8 board</li>
<li>Dark and light squares alternate</li>
<li>12 pieces per side</li>
<li>Pieces can only be placed on dark squares (light squares need to remain empty)</li>
<li>Players alternate turns</li>
<li>Players can only move their own pieces</li>
<li>Players can only move one piece per move</li>
<li>Moves include:</li>
<ul>
<li>Simple move: 1 step diagonally to an unoccupied adjacent square</li>
<li>Jump: 'Jumping' diagonally over an occupied to an unoccupied adjacent square</li>
<li>Jumping leads to the jumped piece being 'captured' and removed from the game</li>
<li>Jumping is mandatory</li>
<li>Multi-jumps are possible and can happen in different directions</li>
<li>Multi-jump sequences need to be completed</li>
<li>Man can only move forward, kings foward and backward</li>
</ul>
<li>Two classes of pieces: Men and kings</li>
<ul>
<li>Men (one piece)</li>
<li>Kings (two pieces stacked on top of each other)</li>
<li>Kings are 'crowned' when pieces have reached the opposite site's kings row</li>
<li>Reaching the opposite site's kings row terminates a move</li>
</ul>
<li>The game ends with a winner when one player has captured all of the other player's pieces</li> 
<li>The game ends with a winner when one player has no more legal moves available.
<li>The game ends in a draw when neither player can achieve a win</li>
</ul>

In [109]:
import copy

In [117]:
def print_state(state):
    for row in state:
        print(row)
    print('\n')

In [168]:
class Game:    
    def __init__(self):
        self.status = "In progress"

        self.state = [
            [0, 'wm', 0, 'wm', 0, 'wm', 0, 'wm'],
            ['wm', 0, 'wm', 0, 'wm', 0, 'wm', 0],
            [0, 'wm', 0, 'wm', 0, 'wm', 0, 'wm'],
            [1, 0, 1, 0, 1, 0, 1, 0],
            [0, 1, 0, 1, 0, 1, 0, 1],
            ['rm', 0, 'rm', 0, 'rm', 0, 'rm', 0],
            [0, 'rm', 0, 'rm', 0, 'rm', 0, 'rm'],
            ['rm', 0, 'rm', 0, 'rm', 0, 'rm', 0]
        ]

    def count_white(self):
        flat_state = [field for row in self.state for field in row]
        return flat_state.count('wm') + 2*flat_state.count('wk')

    def count_red(self):
        flat_state = [field for row in self.state for field in row]
        return flat_state.count('rm') + 2*flat_state.count('rk')

    def get_score(self):
        return self.count_white() - self.count_red()

    def who_won(self):
        score = self.get_score()
        if score > 0:
            return 'White wins'
        elif score < 0:
            return 'Red wins'
        else:
            return "Draw"

    def update_state(self, state):
        if not state:
            self.update_status("Won")
            return
        self.state = state
        self.update_status()

    def update_status(self, status=None):
        if status:
            self.status = status
        if self.count_white == 0 or self.count_red == 0:
            self.status = "Won"


In [169]:
class Player:
    def __init__(self, game, colour):
        self.game = game
        self.colour = colour        
        
    def find_best_move(self, state, depth, alpha, beta, max_player):
        """
        Implements minimax with alpha-beta pruning
        """
        if depth == 0:
            return self.game.get_score(), None
        
        # Maximizing player
        if max_player == True:
            max_evaluation = -1000
            best_next_state = None
            next_possible_states = self._get_next_possible_states(state, True)
            if len(next_possible_states) == 0:
                return max_evaluation, None
            for possible_state in next_possible_states:
                curr_evaluation, _ = self.find_best_move(possible_state, depth - 1, alpha, beta, False)
                max_evaluation = max(max_evaluation, curr_evaluation)
                if max_evaluation == curr_evaluation:
                    best_next_state = possible_state
                alpha = max(alpha, curr_evaluation)
                if beta <= alpha:
                    break
            return max_evaluation, best_next_state
        else:
            # Minimizing player
            min_evaluation = 1000
            best_next_state = None
            next_possible_states = self._get_next_possible_states(state, False)
            if len(next_possible_states) == 0:
                return min_evaluation, None
            for possible_state in next_possible_states:
                curr_evaluation, _ = self.find_best_move(possible_state, depth - 1, alpha, beta, True)
                min_evaluation = min(min_evaluation, curr_evaluation)
                if min_evaluation == curr_evaluation:
                    best_next_state = possible_state
                beta = min(beta, curr_evaluation)
                if beta <= alpha:
                    break
            return min_evaluation, best_next_state

    def _get_next_possible_states(self, state, max_player):
        next_states = []
        for row_id, row in enumerate(state):
            for col_id, field in enumerate(row):
                if field != 0 and field != 1 and self.colour in field:
                    if self.can_make_move('forward', 'left', field, [row_id, col_id], state):
                        next_states.append(self.move('forward', 'left', field, [row_id, col_id], state))
                    if self.can_make_move('forward', 'right', field, [row_id, col_id], state):
                        next_states.append(self.move('forward', 'right', field, [row_id, col_id], state))
                    if self.can_make_move('backward', 'left', field, [row_id, col_id], state):
                        next_states.append(self.move('backward', 'left', field, [row_id, col_id], state))
                    if self.can_make_move('backward', 'right', field, [row_id, col_id], state):
                        next_states.append(self.move('backward', 'right', field, [row_id, col_id], state))

                    if self.can_make_jump('forward', 'left',field, [row_id, col_id], state):
                        next_states.append(self.jump('forward', 'left', field, [row_id, col_id], state))
                    if self.can_make_jump('forward', 'right',field, [row_id, col_id], state):
                        next_states.append(self.jump('forward', 'right', field, [row_id, col_id], state))
                    if self.can_make_jump('backward', 'left',field, [row_id, col_id], state):
                        next_states.append(self.jump('backward', 'left', field, [row_id, col_id], state))
                    if self.can_make_jump('backward', 'right',field, [row_id, col_id], state):
                        next_states.append(self.jump('backward', 'right', field, [row_id, col_id], state))
        return next_states
                        
    def can_make_move(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        if vertical_direction == "backward" and 'k' not in checker:
            return False
            
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
        
        next_checker_row = initial_checker_location[0] + vertical_move
        next_checker_col = initial_checker_location[1] + horizontal_move

        if self.location_is_on_board(next_checker_row, next_checker_col):
            next_location_value = state[next_checker_row][next_checker_col]
            return next_location_value == 1

    def move(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        king_row = 7
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
            king_row = 0
        
        next_checker_row = initial_checker_location[0] + vertical_move
        next_checker_col = initial_checker_location[1] + horizontal_move

        if next_checker_row == king_row:
            checker = checker[0] + 'k'

        next_state = copy.deepcopy(state)
        next_state[next_checker_row][next_checker_col] = checker
        next_state[initial_checker_location[0]][initial_checker_location[1]] = 1 

        return next_state


    def can_make_jump(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        if vertical_direction == "backward" and 'k' not in checker:
            return False
        
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
        
        next_jump_row = initial_checker_location[0] + vertical_move
        next_land_row = initial_checker_location[0] + 2*vertical_move
        next_jump_col = initial_checker_location[1] + horizontal_move
        next_land_col = initial_checker_location[1] + 2*horizontal_move

        if self.location_is_on_board(next_land_row, next_land_col):
            next_jump_value = state[next_jump_row][next_jump_col]
            next_land_value = state[next_land_row][next_land_col]
            return next_jump_value != 1 and not self.colour in next_jump_value and next_land_value == 1

    def jump(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        king_row = 7
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
            king_row = 0
        
        next_jump_row = initial_checker_location[0] + vertical_move
        next_land_row = initial_checker_location[0] + 2*vertical_move
        next_jump_col = initial_checker_location[1] + horizontal_move
        next_land_col = initial_checker_location[1] + 2*horizontal_move

        if next_land_row == king_row:
            checker = checker[0] + 'k'

        next_state = copy.deepcopy(state)
        next_state[next_land_row][next_land_col] = checker
        next_state[next_jump_row][next_jump_col] = 1
        next_state[initial_checker_location[0]][initial_checker_location[1]] = 1
        
        return next_state
        

    def location_is_on_board(self, row, col):
        return row >= 0 and row <= 7 and col >= 0 and col <= 7


In [170]:
game = Game()
player_1 = Player(game, 'w')
player_2 = Player(game, 'r')

current_player = player_1
while game.status != "Won":
    max_player = True if current_player == player_1 else False
    _, next_state = current_player.find_best_move(game.state, 3, -1000, 1000, max_player)
    game.update_state(next_state)
    if current_player == player_1:
        current_player = player_2
    else:
        current_player = player_1

print_state(game.state)
print(game.who_won())
    


[0, 'wm', 0, 'wm', 0, 'wm', 0, 'wm']
['wm', 0, 'wm', 0, 'wm', 0, 'wm', 0]
[0, 'rm', 0, 'rm', 0, 'rm', 0, 1]
['rm', 0, 'rm', 0, 'rm', 0, 'wk', 0]
[0, 'rm', 0, 1, 0, 1, 0, 1]
['rm', 0, 1, 0, 1, 0, 1, 0]
[0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 1, 0]


2
White wins


In [ ]:
# Evaluation functions
state = [] # state representations should who if piece is a man or king
nb_pieces_max = 0
nb_pieces_min = 0

nb_kings_max = 0
nb_kings_min = 0

def get_nb_pieces(state): # check how many pieces Max has left
    return nb_pieces_max

def get_nb_pieces(state): # check how many pieces Max has left
    return nb_pieces_max

In [ ]:
# Simple actions


In [ ]:
# Predicates
def is_in():
    return True
    
def is_king(): # predicte for backward moves
    return True

def reached_king_row(): # predicte for 
    return True

def is_square_empty(): # predicate for move
    return True

def is_jump_possible(): # predictate for move and jump (if a jump is possible, jump rather than move)
    return True

In [ ]:
# Effects
def remove_piece():
    return next_state # with jumped pieces removed

def make_king():
    return next_state # with men that reached kings row as kings

def terminate_move():
    return